In [1]:
import os, re, time, json, math
import datetime
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
import pandas as pd
import google
from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.0


In [2]:
import tensorflow as tf
import tensorflow.keras as keras

from tf_records_generator import get_dataset, count_data_items
from efficientNet_B0 import EffNet0, freeze_blocks
from log_files import save_config, save_fold_iter_history, get_log_dir, save_logs_pickle, generate_columns
from kfold import get_kfold_split, get_repeated_kfold_split
from clr_schedule import CyclicLR
from schedules import LR_test_schedule
from callbacks import CustomStopper

In [3]:
CONFIG = dict(
    log_prefix = "FINAL200_",
    effnet_version = 0,
    input_shape=(256, 256, 3),
    image_resolution=256,
    trainable_base=False,
    time=datetime.datetime.now().strftime("_%d_%m_%Y_%H_%M"),
    use_patient_data=True,
    inner_blocks_frozen=4,

    k_fold = 3,
    kfold_repeats = 5,

    patience = 16,
    min_epochs = 32,

    lr_min=0.000003,
    lr_max=0.0001,
    lr_decay=None,
    clr_step_coefficient = 8,
    
    replicas=8,
    steps_per_epoch=None,
    validation_steps=None,
    batch_size=32,
    epochs=100,
    optimizer='adam',
    loss='binary_crossentropy',
    
    output_bias=np.log([584/32542]),
    weight_for_0 = (1 / 32542)*(32542+584)/2.0,
    weight_for_1 = (1 / 584)*(32542+584)/2.0
)

In [4]:
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()
  data_dir = 'gs://dataset_files/'
else:
  data_dir = 'dataset/'

In [5]:
# Detect hardware
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.TPUStrategy(tpu_resolver)
  print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)
CONFIG["replicas"] = strategy.num_replicas_in_sync


INFO:tensorflow:Initializing the TPU system: grpc://10.28.151.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.28.151.34:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.28.151.34:8470']
Number of accelerators:  8


In [6]:
path = data_dir + str(CONFIG["image_resolution"])
files_train = np.sort(np.array(tf.io.gfile.glob(path + '/train*.tfrec')))
files_test = np.sort(np.array(tf.io.gfile.glob(path + '/test*.tfrec')))


test_ds = get_dataset(files_test, CONFIG)

In [7]:
def make_model(config:dict, strategy)    :
    with strategy.scope():
      model = EffNet0(config=config, trainable_base=config['trainable_base'])

      model.compile(optimizer=keras.optimizers.Adam(), loss=config["loss"], 
                    metrics=[
                                  keras.metrics.TruePositives(name='tp'),
                                  keras.metrics.FalsePositives(name='fp'),
                                  keras.metrics.TrueNegatives(name='tn'),
                                  keras.metrics.FalseNegatives(name='fn'), 
                                  keras.metrics.BinaryAccuracy(name='accuracy'),
                                  keras.metrics.Precision(name='precision'),
                                  keras.metrics.Recall(name='recall'),
                                  keras.metrics.AUC(name='auc'),
                            ]
    )
    return model


In [ ]:
#kfold_split = get_kfold_split(files_train,CONFIG['k_fold'])
kfold_split = get_repeated_kfold_split(files_train,CONFIG['k_fold'], CONFIG['kfold_repeats'])
save_config(CONFIG)
logs = pd.DataFrame()
log_dir = get_log_dir(CONFIG)
for iter, files in kfold_split.items():
    train_ds = get_dataset(files['train'],CONFIG, patient_info=True,repeat=True) 
    val_ds = get_dataset(files['validation'], CONFIG, patient_info=True,repeat=True)
    if CONFIG['steps_per_epoch'] is None:
        CONFIG['steps_per_epoch'] = math.ceil(count_data_items(files["train"])/(CONFIG["batch_size"]*CONFIG["replicas"]))
    if CONFIG['validation_steps'] is None:
        CONFIG['validation_steps'] = math.ceil(count_data_items(files["validation"])/(CONFIG["batch_size"]*CONFIG["replicas"]))

    clr = CyclicLR(base_lr=CONFIG["lr_min"], max_lr=CONFIG["lr_max"],
                                step_size=CONFIG['steps_per_epoch']*CONFIG['clr_step_coefficient'], mode='triangular')
    stopper = CustomStopper(monitor='val_auc',patience=CONFIG['patience'],start_epoch=CONFIG['min_epochs'])

    tb_callback = keras.callbacks.TensorBoard(log_dir=os.path.join(log_dir,f"tb_{iter}"))
    model = make_model(CONFIG, strategy=strategy)
    history = model.fit(train_ds,class_weight={0:CONFIG["weight_for_0"],1:CONFIG["weight_for_1"]},
                        validation_data=val_ds,validation_batch_size=CONFIG["batch_size"]*CONFIG["replicas"],
                        batch_size=CONFIG["batch_size"]*CONFIG["replicas"], epochs=CONFIG["epochs"],
                        steps_per_epoch=CONFIG['steps_per_epoch'],validation_steps=CONFIG['validation_steps'],
                        callbacks=[clr,stopper])
    
    if logs.empty:
        logs = generate_columns(logs,history.history)
    logs = logs.append(history.history, ignore_index=True)
    
save_logs_pickle(logs,CONFIG)


zip_name = os.path.basename(log_dir) + '.zip'
!zip -r $zip_name $log_dir


Epoch 1/100
 2/86 [..............................] - ETA: 5s - loss: 1.9913 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 503.0000 - fn: 9.0000 - accuracy: 0.9824 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4763WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0112s vs `on_train_batch_end` time: 0.1086s). Check your callbacks.


86/86 [==============================] - ETA: 0s - loss: 1.8827 - tp: 2.0000 - fp: 12.0000 - tn: 21616.0000 - fn: 386.0000 - accuracy: 0.9819 - precision: 0.1429 - recall: 0.0052 - auc: 0.5443WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0045s vs `on_test_batch_end` time: 0.0561s). Check your callbacks.


86/86 [==============================] - 24s 284ms/step - loss: 1.8827 - tp: 2.0000 - fp: 12.0000 - tn: 21616.0000 - fn: 386.0000 - accuracy: 0.9819 - precision: 0.1429 - recall: 0.0052 - auc: 0.5443 - val_loss: 0.0897 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 10813.0000 - val_fn: 195.0000 - val_accuracy: 0.9823 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5135
Epoch 2/100
86/86 [==============================] - 15s 173ms/step - loss: 1.4262 - tp: 32.0000 - fp: 813.0000 - tn: 20810.0000 - fn: 361.0000 - accuracy: 0.9467 - precision: 0.0379 - recall: 0.0814 - auc: 0.6499 - val_loss: 0.0847 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 10813.0000 - val_fn: 195.0000 - val_accuracy: 0.9823 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7007
Epoch 3/100
86/86 [==============================] - 15s 170ms/step - loss: 0.9700 - tp: 164.0000 - fp: 4332.0000 - tn: 17291.0000 - fn: 229.0000 - accuracy: 0.7928 - precision: 0.0365 - recall: 0

86/86 [==============================] - ETA: 0s - loss: 2.0540 - tp: 1.0000 - fp: 26.0000 - tn: 21600.0000 - fn: 389.0000 - accuracy: 0.9812 - precision: 0.0370 - recall: 0.0026 - auc: 0.5553WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0047s vs `on_test_batch_end` time: 0.0588s). Check your callbacks.


86/86 [==============================] - 25s 296ms/step - loss: 2.0540 - tp: 1.0000 - fp: 26.0000 - tn: 21600.0000 - fn: 389.0000 - accuracy: 0.9812 - precision: 0.0370 - recall: 0.0026 - auc: 0.5553 - val_loss: 0.0881 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 10815.0000 - val_fn: 193.0000 - val_accuracy: 0.9825 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5843
Epoch 2/100
86/86 [==============================] - 15s 174ms/step - loss: 1.5814 - tp: 23.0000 - fp: 694.0000 - tn: 20928.0000 - fn: 371.0000 - accuracy: 0.9516 - precision: 0.0321 - recall: 0.0584 - auc: 0.6485 - val_loss: 0.1895 - val_tp: 2.0000 - val_fp: 24.0000 - val_tn: 10791.0000 - val_fn: 191.0000 - val_accuracy: 0.9805 - val_precision: 0.0769 - val_recall: 0.0104 - val_auc: 0.6988
Epoch 3/100
86/86 [==============================] - 14s 168ms/step - loss: 1.0296 - tp: 140.0000 - fp: 3886.0000 - tn: 17738.0000 - fn: 252.0000 - accuracy: 0.8120 - precision: 0.0348 - recall: 0.3571 - auc: 0.

...

86/86 [==============================] - 26s 304ms/step - loss: 1.9041 - tp: 0.0000e+00 - fp: 26.0000 - tn: 21601.0000 - fn: 389.0000 - accuracy: 0.9812 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5842 - val_loss: 0.0865 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 10813.0000 - val_fn: 195.0000 - val_accuracy: 0.9823 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6776
Epoch 2/100
86/86 [==============================] - 15s 174ms/step - loss: 1.4350 - tp: 52.0000 - fp: 1244.0000 - tn: 20377.0000 - fn: 343.0000 - accuracy: 0.9279 - precision: 0.0401 - recall: 0.1316 - auc: 0.6318 - val_loss: 0.0894 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 10813.0000 - val_fn: 195.0000 - val_accuracy: 0.9823 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.7058
Epoch 3/100
86/86 [==============================] - 15s 170ms/step - loss: 0.8910 - tp: 171.0000 - fp: 4762.0000 - tn: 16863.0000 - fn: 220.0000 - accuracy: 0.7737 - precision: 0.034

86/86 [==============================] - ETA: 0s - loss: 1.9094 - tp: 0.0000e+00 - fp: 2.0000 - tn: 21627.0000 - fn: 387.0000 - accuracy: 0.9823 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5706WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0049s vs `on_test_batch_end` time: 0.0599s). Check your callbacks.


86/86 [==============================] - 25s 291ms/step - loss: 1.9094 - tp: 0.0000e+00 - fp: 2.0000 - tn: 21627.0000 - fn: 387.0000 - accuracy: 0.9823 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5706 - val_loss: 0.0863 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 10812.0000 - val_fn: 196.0000 - val_accuracy: 0.9822 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6638
Epoch 2/100
86/86 [==============================] - 15s 171ms/step - loss: 1.4960 - tp: 35.0000 - fp: 499.0000 - tn: 21128.0000 - fn: 354.0000 - accuracy: 0.9613 - precision: 0.0655 - recall: 0.0900 - auc: 0.6651 - val_loss: 0.0946 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 10812.0000 - val_fn: 196.0000 - val_accuracy: 0.9822 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6642
Epoch 3/100
86/86 [==============================] - 15s 169ms/step - loss: 1.0105 - tp: 135.0000 - fp: 3216.0000 - tn: 18409.0000 - fn: 256.0000 - accuracy: 0.8423 - precision: 0.0403 

86/86 [==============================] - ETA: 0s - loss: 2.0174 - tp: 1.0000 - fp: 9.0000 - tn: 21616.0000 - fn: 390.0000 - accuracy: 0.9819 - precision: 0.1000 - recall: 0.0026 - auc: 0.5385WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0045s vs `on_test_batch_end` time: 0.0583s). Check your callbacks.


86/86 [==============================] - 25s 294ms/step - loss: 2.0174 - tp: 1.0000 - fp: 9.0000 - tn: 21616.0000 - fn: 390.0000 - accuracy: 0.9819 - precision: 0.1000 - recall: 0.0026 - auc: 0.5385 - val_loss: 0.0856 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 10816.0000 - val_fn: 192.0000 - val_accuracy: 0.9826 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6621
Epoch 2/100
86/86 [==============================] - 15s 177ms/step - loss: 1.6001 - tp: 22.0000 - fp: 507.0000 - tn: 21114.0000 - fn: 373.0000 - accuracy: 0.9600 - precision: 0.0416 - recall: 0.0557 - auc: 0.6455 - val_loss: 0.0838 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 10816.0000 - val_fn: 192.0000 - val_accuracy: 0.9826 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6734
Epoch 3/100
86/86 [==============================] - 14s 167ms/step - loss: 1.0812 - tp: 134.0000 - fp: 3002.0000 - tn: 18622.0000 - fn: 258.0000 - accuracy: 0.8519 - precision: 0.0427 - recall: 0.

In [ ]:
print("jacek")

In [ ]:
log_dir = get_log_dir(CONFIG)
zip_name = os.path.basename(log_dir) + '.zip'
!zip -r $zip_name $log_dir

In [ ]:
i = 2
plt.plot(logs['auc'][i])
plt.plot(logs['val_auc'][i])
suma = 0
for k in [0,1,2]:
 suma += np.max(logs['val_auc'][k])
suma/3
np.max(logs['val_auc'][i])